## UseCase 2 - Video Transcribe

**References**

1. [Transribe - Python](https://docs.aws.amazon.com/code-library/latest/ug/python_3_transcribe_code_examples.html)

In [ ]:
import configparser
import time
import boto3
import json
import logging
import urllib.request
import random
from botocore.exceptions import ClientError
from typing import Dict, List

### Initiate Services

In [ ]:
logging.basicConfig(
    format="%(levelname)s - %(asctime)s - %(message)s", level=logging.ERROR
)

logger = logging

### Reading Config File

In [ ]:
CONFIG_FILENAME=".config.ini"

config = configparser.ConfigParser()
config.read(CONFIG_FILENAME)

def get_value_by_section_and_key(section, key):
        """get_value_by_section_and_key"""
        return config.get(section, key)

def get_all_details_of_section(section) -> dict:
    """get_all_details_of_section"""
    return dict(config.items(section))


default_cfgs = get_all_details_of_section("DEFAULT")

### Create boto3 client - BEDROCK

In [ ]:
transcribe_client = boto3.client(
    'transcribe',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)


### Transcribe File

In [ ]:
def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={"MediaFileUri": file_uri},
        MediaFormat="mp4",
        LanguageCode="en-US",
    )

    max_tries = 60
    while max_tries > 0:
        transcript_url = ""
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job["TranscriptionJob"]["TranscriptionJobStatus"]
        if job_status in ["COMPLETED", "FAILED"]:
            print(f"Job {job_name} is {job_status}.")
            if job_status == "COMPLETED":
                print(
                    f"Download the transcript from\n"
                    f"\t{job['TranscriptionJob']['Transcript']['TranscriptFileUri']}."
                )
                transcript_url = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)

    text = ""
    if(len(transcript_url) > 0):
        response = urllib.request.urlopen(transcript_url)
        data = json.loads(response.read())
        
        # Extract the transcript text
        text = data['results']['transcripts'][0]['transcript']

    return text

BUCKET_NAME="tr-hackathon"
MEDIAFILE="usecase-exp/nirmala-sitharaman-fm-parliament.mp4"
file_uri = "s3://" + BUCKET_NAME + "/" + MEDIAFILE
job_id = "Transcribe-job-"+str(random.randint(9, 10054))
transcibed_text = transcribe_file(job_id, file_uri, transcribe_client)

transcibed_text

### Translate Text

1. [List of Supported Lanugages](https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html)

In [ ]:
translate_client = boto3.client(
    'translate',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)


transalated_text = translate_client.translate_text(Text=transcibed_text, 
            SourceLanguageCode="en", TargetLanguageCode="fr")
print('TranslatedText: ' + transalated_text.get('TranslatedText'))
print('SourceLanguageCode: ' + transalated_text.get('SourceLanguageCode'))
print('TargetLanguageCode: ' + transalated_text.get('TargetLanguageCode'))

### Summarize

In [ ]:
# def split_text_into_chunks(text: str, max_length: int = 4900) -> List[str]:
#     """
#     Split text into chunks smaller than Amazon Comprehend's limit
#     """
#     words = text.split()
#     chunks = []
#     current_chunk = []
#     current_length = 0
    
#     for word in words:
#         # Add space before word (except for first word in chunk)
#         word_length = len(word) + (1 if current_chunk else 0)
        
#         if current_length + word_length <= max_length:
#             current_chunk.append(word)
#             current_length += word_length
#         else:
#             # Save current chunk and start new one
#             chunks.append(' '.join(current_chunk))
#             current_chunk = [word]
#             current_length = len(word)
    
#     # Add the last chunk
#     if current_chunk:
#         chunks.append(' '.join(current_chunk))
    
#     return chunks

# def analyze_sentiment(text: str) -> Dict:
#     """
#     Analyze sentiment of text chunk
#     """
#     try:
#         comprehend_client = boto3.client(
#             'comprehend',
#             region_name=default_cfgs.get("aws_default_region", ""),
#             aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
#             aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
#             aws_session_token=default_cfgs.get("aws_session_token", "")
#         )
#         response = comprehend_client.detect_sentiment(
#             Text=text,
#             LanguageCode='en'
#         )
#         print(response)
#         return {
#             'sentiment': response['Sentiment'],
#             'scores': response['SentimentScore']
#         }
#     except Exception as e:
#         print(f"Error analyzing sentiment: {str(e)}")
#         return None

# def get_key_phrases(text: str) -> List[str]:
#     """
#     Extract key phrases from text chunk
#     """
#     try:
#         comprehend_client = boto3.client(
#             'comprehend',
#             region_name=default_cfgs.get("aws_default_region", ""),
#             aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
#             aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
#             aws_session_token=default_cfgs.get("aws_session_token", "")
#         )
#         response = comprehend_client.detect_key_phrases(
#             Text=text,
#             LanguageCode='en'
#         )
#         return [phrase['Text'] for phrase in response['KeyPhrases']]
#     except Exception as e:
#         print(f"Error extracting key phrases: {str(e)}")
#         return []

# def summarize_text(text: str) -> Dict:
#     """
#     Summarize text while handling size limitations
#     """
#     # Split text into chunks
#     chunks = split_text_into_chunks(text)
    
#     # Initialize results
#     all_sentiments = []
#     all_key_phrases = []
    
#     # Process each chunk
#     for i, chunk in enumerate(chunks):
#         print(f"Processing chunk {i+1}/{len(chunks)}")
        
#         # Analyze sentiment
#         sentiment_result = analyze_sentiment(chunk)
#         if sentiment_result:
#             all_sentiments.append(sentiment_result)
            
#         # Get key phrases
#         key_phrases = get_key_phrases(chunk)
#         all_key_phrases.extend(key_phrases)
        
#         # Add small delay to avoid rate limiting
#         time.sleep(0.1)
    
#     # Combine results
#     if all_sentiments:
#         # Calculate overall sentiment
#         sentiment_counts = {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0, 'MIXED': 0}
#         total_scores = {'Positive': 0.0, 'Negative': 0.0, 'Neutral': 0.0, 'Mixed': 0.0}
        
#         for result in all_sentiments:
#             sentiment_counts[result['sentiment']] += 1
#             for key, value in result['scores'].items():
#                 total_scores[key] += value
        
#         # Calculate averages
#         num_chunks = len(all_sentiments)
#         avg_scores = {key: value/num_chunks for key, value in total_scores.items()}
#         overall_sentiment = max(sentiment_counts.items(), key=lambda x: x[1])[0]
        
#         # Get unique key phrases
#         unique_phrases = list(dict.fromkeys(all_key_phrases))
        
#         return {
#             'status': 'success',
#             'overall_sentiment': overall_sentiment,
#             'sentiment_scores': avg_scores,
#             'key_phrases': unique_phrases[:10],  # Top 10 key phrases
#             'chunks_processed': len(chunks)
#         }
    
#     return {
#         'status': 'error',
#         'message': 'No results generated'
#     }

# def generate_summary_text(analysis_result: Dict) -> str:
#     """
#     Generate readable summary from analysis results
#     """
#     if analysis_result['status'] != 'success':
#         return "Could not generate summary."
    
#     summary = []
    
#     # Add sentiment information
#     summary.append(f"Overall Sentiment: {analysis_result['overall_sentiment']}")
#     summary.append("\nSentiment Scores:")
#     for sentiment, score in analysis_result['sentiment_scores'].items():
#         summary.append(f"- {sentiment}: {score:.2%}")
    
#     # Add key phrases
#     summary.append("\nKey Points:")
#     for phrase in analysis_result['key_phrases']:
#         summary.append(f"- {phrase}")

#     print(analysis_result)
#     print("********")
    
#     return '\n'.join(summary)

# # Get analysis results
# print("Analyzing text...")
# results = summarize_text(transcibed_text)

# # Generate and print summary
# print("\nSummary:")
# print(generate_summary_text(results))

# # Print processing statistics
# print(f"\nProcessed {results.get('chunks_processed', 0)} chunks of text")

In [ ]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=default_cfgs.get("aws_default_region", ""),
    aws_access_key_id=default_cfgs.get("aws_access_key_id", ""),
    aws_secret_access_key=default_cfgs.get("aws_secret_access_key", ""),
    aws_session_token=default_cfgs.get("aws_session_token", "")
)

system_prompt = "Give a text of transcribed text of a video, generate a concise summary of 80-120 words. \
Ensure to not miss out on any important points. \
The input text can be in any language, and the summary has to be returned in the same language. \
Return the response containing only the summary."

payload = {
    "modelId": "anthropic.claude-3-5-sonnet-20241022-v2:0",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4000,
        "messages": [
            {
                "role": "user",
                "content": [
                {
                    "type": "text",
                    "text": transalated_text.get('TranslatedText')
                }
                ]
            }
        ],
        "system": system_prompt
    }
}
# print(kwargs)

response = bedrock_runtime.invoke_model(
        modelId=payload["modelId"],
        body=json.dumps(payload["body"]).encode('utf-8')
    )
response_body = json.loads(response.get('body').read())
response_body